# Hoh River tiles

## In collaboration with the Hoh Indian Tribe

Under development for the [Cascadia CoPes Hub](https://cascadiacopeshub.org/) project, supported by NSF.

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
from clawpack.geoclaw import topotools
from clawpack.visclaw import geoplot
from importlib import reload
import geopandas as gpd
import folium
import pandas as pd
import merge_topos

In [ ]:
import find_topo_source

In [ ]:
eighth = 1/8  # half tile width
yrange = arange(47.75, 48.00+eighth, 0.25)
xrange = arange(-124.5, -124.5+eighth, 0.25)
name = 'NCEI tiles around Hoh River'
find_topo_source.make_combined_tile_kml(name, xrange, yrange)

In [ ]:
tile_names = []
for x in xrange:
    for y in yrange:
        xm = x+eighth
        ym = y-eighth
        tile_name = find_topo_source.tile_coords(xm,ym,verbose=False)
        print(f'midpoints: {xm:.3f}, {ym:.3f}, NW corner:  {x:.3f}, {y:.3f}, tile: {tile_name}')
        tile_names.append(tile_name)

In [ ]:
gdf = gpd.read_file('NCEI_tiles_around_Hoh_River.kml', driver='KML')
m = folium.Map(location=(47.75,-124.43), zoom_start=10, scrollWheelZoom=False)
folium.GeoJson(gdf).add_to(m)

for x in xrange:
    for y in yrange:
        xm = x+eighth
        ym = y-eighth
        tile_name = find_topo_source.tile_coords(xm,ym,verbose=False)
        folium.Marker(
            location=[y-eighth,x+eighth],
            #popup = f"{x:.3f}, {y:.3f}: \n<b>Tile:</b>\n {tile_name}",
            popup = f"<b>Tile:</b>\n {tile_name}",
            tooltip="Click for info",
            icon=folium.Icon(color="red")  #, icon="cloud") # Customize the marker's appearance
        ).add_to(m) 
m

In [ ]:
for tile_name in tile_names:
    print('\n-----------------------------------')
    tile_urls = find_topo_source.find_tile_url(tile_name, verbose=True)
    if len(tile_urls) == 0:
        print(f'{tile_name} not found')

In [ ]:
def read_tif(url, print_warnings=False):
    import rasterio
    import warnings
    
    with warnings.catch_warnings(record=True) as w:
        with rasterio.open(url) as src:
            print(f"Coordinate Reference System (CRS): {src.crs}")
            print(f"Bounds: {src.bounds}")
            #print(f"Number of bands: {src.count}") # should always be 1
            bounds = src.bounds
            print(f"Reading Z data array with shape {src.width}x{src.height}...")
            topotif = src.read()
            if print_warnings:
                print('topofile read with following warnings:')
                print(f'+++ len(w) = {len(w)}')
                for warning in w:
                    print(f"- Category: {warning.category.__name__}, Message: {str(warning.message)}")
                    
    Z = flipud(topotif[0,:,:])
    x1,x2,y1,y2 = extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]
    dx = (x2-x1)/Z.shape[0]
    dy = (y2-y1)/Z.shape[1]
    print(f'dx = {dx:.7f} degrees = {dx*3600:.7f} arcseconds')
    print(f'dy = {dy:.7f} degrees = {dy*3600:.7f} arcseconds')
    
    x = arange(x1+0.5*dx, x2, dx)
    y = arange(y1+0.5*dy, y2, dy)
    
    topo = topotools.Topography()
    topo.set_xyZ(x,y,Z)

    return topo

In [ ]:
url = 'https://coast.noaa.gov/htdata/raster2/elevation/NCEI_ninth_Topobathy_2014_8483/wash_outercoast/ncei19_n47x75_w124x50_2025v2.tif'
tile_n47x75_w124x50_2025v2 = read_tif(url)

In [ ]:
url = 'https://coast.noaa.gov/htdata/raster2/elevation/NCEI_ninth_Topobathy_2014_8483/wash_outercoast/ncei19_n48x00_w124x50_2025v2.tif'
tile_n48x00_w124x50_2025v2 = read_tif(url)

## Shift from NAVD88 to MHW

Note: Difference at:

    Wesport tide gauge: 2.22 m
    La Push tide gauge: 1.94 m

In [ ]:
dz_mhw = 2.0  # approximate correction NAVD88 to MHW

tile_navd88 = tile_n48x00_w124x50_2025v2
tile_n48x00_w124x50_2025v2_mhw = topotools.Topography()
tile_n48x00_w124x50_2025v2_mhw.set_xyZ(tile_navd88.x, tile_navd88.y, tile_navd88.Z - dz_mhw)

tile_navd88 = tile_n47x75_w124x50_2025v2
tile_n47x75_w124x50_2025v2_mhw = topotools.Topography()
tile_n47x75_w124x50_2025v2_mhw.set_xyZ(tile_navd88.x, tile_navd88.y, tile_navd88.Z - dz_mhw)

In [ ]:
extent = [-124.46, -124.32, 47.73, 47.76]
Hoh1 = tile_n47x75_w124x50_2025v2_mhw.crop(extent)
Hoh2 = tile_n48x00_w124x50_2025v2_mhw.crop(extent)

In [ ]:
fig,ax = subplots(figsize=(10,8))
Hoh1.plot(axes=ax, limits=(-50,50), add_colorbar=False)
Hoh2.plot(axes=ax, limits=(-50,50), cb_kwargs={'shrink':0.3, 'extend':'both'})
axis(extent)

In [ ]:
extent2 = [-124.442, -124.42, 47.745, 47.755]
fig,ax = subplots(figsize=(10,8))
Hoh1.plot(axes=ax, limits=(-10,10), add_colorbar=False)
Hoh2.plot(axes=ax, limits=(-10,10), cb_kwargs={'shrink':0.3, 'extend':'both'})
plot(extent2[:2], [47.75, 47.75], 'k--', label='tile edges')
legend(loc='upper right', framealpha=1)
axis(extent2)
title('Hoh River mouth');

In [ ]:
Hoh1.y[-3:], Hoh2.y[:3]

In [ ]:
(47.75001543 - 47.74998457)*9*3600

## Missing data

Data is missing on the southern part of this tile at the request of the Quinault Nation.

In [ ]:
tile_n47x75_w124x50_2025v2.Z.min()

In [ ]:
tile_n47x75_w124x50_2025v2.crop(coarsen=18).plot(limits=(-100,100))

In [ ]:
missing = where(tile_n47x75_w124x50_2025v2.Z == -9999.)
Ymissing = tile_n47x75_w124x50_2025v2.Y[missing[0],missing[1]]
ymissing = tile_n47x75_w124x50_2025v2.y[missing[0]]
print(f'Missing data south of y = {Ymissing.max():.4f} = {ymissing.max():.4f}')

## Merge the two tiles

In [ ]:
assert abs(tile_n47x75_w124x50_2025v2_mhw.x - tile_n48x00_w124x50_2025v2_mhw.x).max() < 1e-10, \
        '*** x arrays should agree'
dy = 1/(9*3600)
dy_edge = tile_n48x00_w124x50_2025v2_mhw.y[0] - tile_n47x75_w124x50_2025v2_mhw.y[-1]
assert abs(dy_edge - dy) < 0.01*dy, \
       f'*** y arrays should be adjacent with gap {dy}, dy_edge = {dy_edge}'

x = tile_n47x75_w124x50_2025v2_mhw.x
y = hstack((tile_n47x75_w124x50_2025v2_mhw.y, tile_n48x00_w124x50_2025v2_mhw.y))

dys = diff(y)
print(f'y has {len(y)} points with min dy = {dys.min()}, max dy = {dys.max()}')

In [ ]:
X,Y = meshgrid(x,y)
Z = zeros(X.shape)
topo = topotools.Topography()
topo.set_xyZ(x,y,Z)
print('New topo extent: ', topo.extent)

In [ ]:
name = 'Hoh_mergedtiles_19s_mhw'
topo = merge_topos.overwrite(topo, tile_n47x75_w124x50_2025v2_mhw)
merge_topos.plot_topos(topo, tile_n47x75_w124x50_2025v2_mhw,
                       'topo','tile_n47x75_w124x50_2025v2_mhw',
                       coarsen=18, limits=(-100,100), buffer=0.02)

In [ ]:
topo = merge_topos.overwrite(topo, tile_n48x00_w124x50_2025v2_mhw)
merge_topos.plot_topos(topo, tile_n48x00_w124x50_2025v2_mhw,
                       'topo','tile_n48x00_w124x50_2025v2_mhw',
                       coarsen=18, limits=(-100,100), buffer=0.02)

In [ ]:
#extent_save = [-124.5, -124.35, 47.65, 47.81]
extent_save = [-124.44, -124.39, 47.73, 47.76]
topo_save = topo.crop(extent_save)
fname = 'HohRiver_19s_mhw.asc'
topo_save.write(fname, 3, header_style='asc', Z_format='%.3f')
print(f'Created {fname} with Z.shape = {topo_save.Z.shape}')

In [ ]:
ls -lh HohRiver_19s_mhw.asc

In [ ]:
topo_save.crop(coarsen=3).plot(limits=(-100,100), cb_kwargs={'extend':'both','shrink':0.5})
title(f'{fname}\ncoarsened by 3');
fname_png = fname.replace('asc','png')
savefig(fname_png)
print('Created ',fname_png)

## Larger version covering Kalaloch (coarsened to 1/3")

In [ ]:
extent_HohKalaloch = [-124.5, -124.35, 47.57, 47.81]
topo_HohKalaloch = topo.crop(extent_HohKalaloch, coarsen=3)
fname = 'HohKalaloch_13s_mhw.asc'
topo_HohKalaloch.write(fname, 3, header_style='asc', Z_format='%.3f')
print(f'Created {fname} with Z.shape = {topo_save.Z.shape}')

In [ ]:
ls -lh HohKalaloch*

In [ ]:
fig,ax = subplots(figsize=(8,10))
topo_HohKalaloch.plot(axes=ax,limits=(-100,100), cb_kwargs={'extend':'both','shrink':0.7})
title(f'{fname}');
fname_png = fname.replace('asc','png')
savefig(fname_png)
print('Created ',fname_png)

## Create coarsened 1s versions

In [ ]:
n48x00_w124x50_1s = tile_n48x00_w124x50_2025v2_mhw.crop(coarsen=9, align=(-124.5,47.5))

In [ ]:
fname = 'n48x00_w124x50_1s.asc'
n48x00_w124x50_1s.write(fname, 3, header_style='asc', Z_format='%.3f')
print(f'Created {fname}')

In [ ]:
n48x00_w124x50_1s.plot(limits=(-100,100), cb_kwargs={'extend':'both','shrink':0.5})
title('n48x00_w124x50_1s');
fname_png = 'nn48x00_w124x50_1s.png'
savefig(fname_png)
print('Created ',fname_png)

### Crop southern tile to remove region with no data

In [ ]:
extent = [-125, -124, 47.565, 48]
n47x75_w124x50_cropped_1s = tile_n47x75_w124x50_2025v2.crop(extent, coarsen=9, align=(-124.5,47.5))

In [ ]:
print(f'cropped 1s file has shape {n47x75_w124x50_cropped_1s.Z.shape}' \
    + f'and starts at y = {n47x75_w124x50_cropped_1s.y[0]:.4f}')

In [ ]:
fname = 'n47x75_w124x50_cropped_1s.asc'
n47x75_w124x50_cropped_1s.write(fname, 3, header_style='asc', Z_format='%.3f')
print(f'Created {fname}')

In [ ]:
n47x75_w124x50_cropped_1s.plot(limits=(-100,100), cb_kwargs={'extend':'both','shrink':0.5})
title('n47x75_w124x50_cropped_1s \nCropped to avoid nodata region');
fname_png = 'n47x75_w124x50_cropped_1s.png'
savefig(fname_png)
print('Created ',fname_png)